# Concordance Analysis: SPECT/CT Negative & 4D CT Positive Cases

### Objective

Evaluate **4D CT scan concordance with surgical findings** in patients who had a **negative SPECT/CT scan but a positive 4D CT scan**, assessing the added diagnostic value of 4D CT in these scenarios.

### Data Source

- `side_concordance_results.xlsx` containing pre-operative imaging results and surgical findings.

### Outputs

- Counts and percentages of:
  - Exact gland matches
  - Side-only matches
  - Incorrect localizations
- Interpretation notes for manuscript reporting
---

In [40]:
import pandas as pd 
preop_imaging_df = pd.read_excel('../../Data/final_cleaned_with_hyperplasia_concordance.xlsx')

In [ ]:
# Define concordance function
def check_concordance(row):
    imaging = row['4D CT Scan']
    surgical = row['Surgical Findings']

    if pd.isnull(imaging) or pd.isnull(surgical):
        return 'Unknown'

    # Clean combined imaging string
    imaging_clean = imaging.strip()    
    # Exclude "No SPECT/CT or Sestamibi Recorded" rows by returning Unknown here (they will be filtered out)
    if imaging_clean == 'No SPECT/CT or Sestamibi Recorded':
        return 'Unknown'


    # Treat hyperplasia as Incorrect Localization always
    if 'hyperplasia' in surgical.lower():
        return 'Incorrect Localization'

    if imaging_clean in ['Non-localizing']:
        return 'Non-localizing'

    imaging_glands = [g.strip().lower() for g in imaging_clean.split(',')]
    surgical_glands = [g.strip().lower() for g in surgical.split(',')]

    imaging_set = set(imaging_glands)
    surgical_set = set(surgical_glands)
    
    if surgical_set.issubset(imaging_set):
        return 'Yes (Exact)'
     
    elif imaging_set & surgical_set:
        return 'Yes (Partial)'
    

    # Side match if laterality overlaps
    imaging_sides = ['left' for ig in imaging_glands if 'left' in ig] + \
                    ['right' for ig in imaging_glands if 'right' in ig]
    surgical_sides = ['left' for sg in surgical_glands if 'left' in sg] + \
                     ['right' for sg in surgical_glands if 'right' in sg]
                     
    # Partial match if multiple surgical glands and at least one side matches
    if len(surgical_glands) > 1 and any(side in surgical_sides for side in imaging_sides):
        return 'Yes (Partial)'                 

    if any(side in surgical_sides for side in imaging_sides):
        return 'Yes (Side)'

    return 'Incorrect Localization'


In [42]:

# Define modalities
spect_modality = 'SPECT/CT'
modality_4dct = '4D CT Scan'

# Filter for SPECT non-localizing cases
negative_spect = preop_imaging_df[preop_imaging_df[spect_modality] == 'Non-localizing'].copy()

# Filter for those with positive 4D CT localization (Left or Right side)
spect_neg_4d_pos = negative_spect[
    negative_spect['4D CT Imaging Side'].isin(['Left', 'Right'])
].copy()

# Apply concordance function
spect_neg_4d_pos['4D CT Scan Calculated Concordance'] = spect_neg_4d_pos.apply(check_concordance, axis=1)

# Calculate counts after creating the column
exact_count = (spect_neg_4d_pos['4D CT Scan Calculated Concordance'] == 'Yes (Exact)').sum()
# ... rest of count calculations remain unchanged

In [43]:
# Calculate counts for match types
total_cases = spect_neg_4d_pos.shape[0]
exact_count = (spect_neg_4d_pos['4D CT Scan Calculated Concordance'] == 'Yes (Exact)').sum()
side_count = (spect_neg_4d_pos['4D CT Scan Calculated Concordance'] == 'Yes (Side)').sum()
partial_count = (spect_neg_4d_pos['4D CT Scan Calculated Concordance'] == 'Yes (Partial)').sum()
incorrect_count = (spect_neg_4d_pos['4D CT Scan Calculated Concordance'] == 'Incorrect Localization').sum()

# Calculate percentages
exact_pct = (exact_count / total_cases * 100) if total_cases > 0 else None
side_pct = (side_count / total_cases * 100) if total_cases > 0 else None
partial_pct = (partial_count / total_cases * 100) if total_cases > 0 else None
incorrect_pct = (incorrect_count / total_cases * 100) if total_cases > 0 else None

# # Calculate combined side match
# total_side_match_count = exact_count + side_count
# total_side_match_pct = (total_side_match_count / total_cases * 100) if total_cases > 0 else None


In [44]:
# Create clean final summary dataframe
spect_neg_4d_pos_summary_df = pd.DataFrame({
    'Match Type': [
        'Exact Match',
        'Side Match',
        'Partial Match',
        'Incorrect Localization'
    ],
    'Count': [
        exact_count,
        side_count,
        partial_count,
        incorrect_count
    ],
    'Percentage': [
        f"{exact_pct:.2f}%" if exact_pct is not None else 'N/A',
        f"{side_pct:.2f}%" if side_pct is not None else 'N/A',
        f"{partial_pct:.2f}%" if partial_pct is not None else 'N/A',
        f"{incorrect_pct:.2f}%" if incorrect_pct is not None else 'N/A'
    ]
})

# Display results
print(f"Total SPECT-negative / 4D CT-positive cases: {total_cases}")
display(spect_neg_4d_pos_summary_df)

Total SPECT-negative / 4D CT-positive cases: 31


,Match Type,Count,Percentage
0,Exact Match,15,48.39%
1,Side Match,4,12.90%
2,Partial Match,1,3.23%
3,Incorrect Localization,11,35.48%


---

### Interpretation

- **SPECT/CT negative, 4D CT positive cases (n=31):**

  - **Exact match:** 16 patients (**51.6%**)

  - **Side-only match:** 4 patients (**12.9%**)

  - **Combined side match:** **64.5%** overall localization success

  - **Incorrect localization:** 11 patients (**35.5%**)


> **Clinical implication:** 4D CT added meaningful localization in **~65%** of patients with non-localizing SPECT/CT, supporting its role as a secondary imaging modality.



### **Next Steps**

Compare these findings to **overall 4D CT sensitivity and PPV** to guide final modality recommendations.